## 1. Modified CAPM


In [1]:
#Import packages
import yfinance as yf
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#import sys
#!{sys.executable} -m pip install openpyxl

E(Ri)=rf+βi×[E(RM)−rf]+CR

In [3]:
#Import RF from Fama French database
FF=pd.read_csv('F-F_Research_Data_Factors.CSV', header=1148,index_col=0).dropna()
FF.index = pd.to_datetime(FF.index)#,format='%Y')
FF.head()

,Mkt-RF,SMB,HML,RF
1927-01-01,29.47,-2.83,-3.40,3.12
1928-01-01,35.39,4.51,-6.17,3.56
1929-01-01,-19.54,-30.69,11.65,4.75
1930-01-01,-31.23,-5.34,-11.76,2.41
1931-01-01,-45.11,3.64,-14.56,1.07


In [4]:
#Import Damodaran's database
Damodaran=pd.ExcelFile('ctryprem.xlsx')
RWA_tab= pd.read_excel(Damodaran, 'Regional Weighted Averages',header=0,index_col=0)
RWA_tab.head()

,GDP (in billions),Moody's rating,Adj. Default Spread,Equity Risk Premium,Country Risk Premium,Corporate Tax Rate,Region,GDP Weight,Weight*Default Spread,Weight*ERP,Weight*CRP,Weight * Tax Rate
Country,,,,,,,,,,,,
Angola,62.307,B3,0.055345,0.106725,0.064325,0.25,Africa,0.030376,0.001681,0.003242,0.001954,0.007594
Benin,15.652,B1,0.038304,0.086919,0.044519,0.3,Africa,0.007631,0.000292,0.000663,0.000340,0.002289
Botswana,15.782,A3,0.010209,0.054266,0.011866,0.22,Africa,0.007694,0.000079,0.000418,0.000091,0.001693
Burkina Faso,17.369,B2,0.046824,0.096822,0.054422,0.28,Africa,0.008468,0.000397,0.00082,0.000461,0.002371
Cameroon,39.802,B2,0.046824,0.096822,0.054422,0.33,Africa,0.019405,0.000909,0.001879,0.001056,0.006404


In [5]:
# Get the risk free return and excess market return
year=input('Please input the year (1927-2020) for risk free interest rate & excess market return then press Enter: ')
Rf=FF.loc[year].RF
Excess_Rm=FF.loc[year]['Mkt-RF']
print('In {}, the Risk-free interest is {}%, and the Excess market return is {}%'.format(year,Rf.values[0],Excess_Rm.values[0]))

Please input the year (1927-2020) for risk free interest rate & excess market return then press Enter: 2020
In 2020, the Risk-free interest is 0.44%, and the Excess market return is 23.67%


In [6]:
# Get stock beta
def beta(stock):
    return yf.Ticker(stock).info['beta']
def beta1Y(stock,year=year):
    asset=yf.download(stock, 
                      start='{}-01-01'.format(year), 
                      end='{}-06-12'.format(year))['Adj Close']
    market=yf.download('SPY', 
                      start='{}-01-01'.format(year), 
                      end='{}-06-12'.format(year))['Adj Close']
    ols = sm.OLS(asset,market-Rf[0]).fit()

    return ols.params[0]
    

ref_ticker=input('Please input the reference stock for beta calculation then press Enter: ')
beta_i=beta(ref_ticker)
print('The current beta of {} is {}'.format(ref_ticker,beta_i))
beta_y=beta1Y(ref_ticker)
print('The beta in year {} of {} is {}'.format(year,ref_ticker,beta_y))

Please input the reference stock for beta calculation then press Enter: AAPL
The current beta of AAPL is 1.202736
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The beta in year 2020 of AAPL is 0.2524801893243005


In [8]:
# Get Country Risk
country=input('Please input country of the interested stock then press Enter: ')
CR=RWA_tab.loc[country,'Country Risk Premium']
print('The country risk premium of {} is {}'.format(country,CR))

Please input country of the interested stock then press Enter: United States
The country risk premium of United States is 0


In [9]:
#Calculate modified CAPM
Mod_R=Rf+beta_y*Excess_Rm+CR*100
print('The modified CAPM expected rate of return is {}%'.format(Mod_R.values[0]))

The modified CAPM expected rate of return is 6.416206081306194%


## Modified International CAPM

In [11]:
#Get beta_mM and beta_nM
beta_iM=[]
countries=[]
while True:
    country_name=input('Please input country of interest then press Enter to proceed or Q to end: ')
    countries.append(country_name)
    if countries[-1] !='Q':
        beta_iM.append(1+RWA_tab.loc[countries[-1],'Equity Risk Premium'])
    else:
        break


Please input country of interest then press Enter to proceed or Q to end: Indonesia
Please input country of interest then press Enter to proceed or Q to end: Chile
Please input country of interest then press Enter to proceed or Q to end: Q


In [12]:
beta_iM

[1.0612246574388315, 1.04935887810535]

In [14]:
#Get beta_tM
def beta_sect(SPDR_sectorETF):
    return yf.Ticker(SPDR_sectorETF).info['beta3Year']

sector=input('Please input sector of interest then press Enter: ')
beta_tM=beta_sect(sector)
print('The beta of {} is {}'.format(sector,beta_tM))


Please input sector of interest then press Enter: XLF
The beta of XLF is 1.25


In [15]:
#Calculate beta_tnM and beta_tmM
beta_tiM=[beta_tM*element for element in beta_iM]

In [16]:
#What is the alpha factorsssss? I'll will just assume it is a dirichlet random variables

alphas=np.random.dirichlet(np.ones(len(beta_tiM)))
alphas

array([0.63115471, 0.36884529])

In [17]:
#Calculate beta_p
beta_p=alphas@np.array(beta_tiM)
beta_p

1.3210600258291363

In [18]:
#Calculate modified international CAMP
Int_R=Rf+beta_p*Excess_Rm
print('The modified international CAPM expected rate of return is {}%'.format(Int_R.values[0]))

The modified international CAPM expected rate of return is 31.70949081137566%
